In [12]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report,precision_score, recall_score, f1_score

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
import numpy as np

In [13]:
# Load and preprocess data

train_data = pd.read_csv('/Users/kannanpavanaa/Downloads/archive (5)/loan_sanction_test.csv')
test_data = pd.read_csv('/Users/kannanpavanaa/Downloads/archive (5)/loan_sanction_test.csv')



In [14]:
#Handle missing values for training data

numeric_columns_train = train_data.select_dtypes(include=['float64', 'int64']).columns
train_data[numeric_columns_train] = train_data[numeric_columns_train].fillna(train_data[numeric_columns_train].median())


categorical_columns_train = train_data.select_dtypes(include=['object']).columns
for col in categorical_columns_train:
    train_data[col].fillna(train_data[col].mode()[0], inplace=True)

In [15]:
#Handle missing values for testing data

numeric_columns_test = test_data.select_dtypes(include=['float64', 'int64']).columns
test_data[numeric_columns_test] = test_data[numeric_columns_test].fillna(test_data[numeric_columns_test].median())

categorical_columns_test = test_data.select_dtypes(include=['object']).columns
for col in categorical_columns_test:
    test_data[col].fillna(test_data[col].mode()[0], inplace=True)

In [16]:
# Encoding of categorical variables in both training and testing data
label_encoders = {}
categorical_columns = ['Gender', 'Married', 'Education', 'Self_Employed', 'Property_Area']

for col in categorical_columns:
    le = LabelEncoder()
    train_data[col] = le.fit_transform(train_data[col])
    test_data[col] = le.transform(test_data[col])  # Ensure you apply the same encoding for the test set
    label_encoders[col] = le

#Clean the 'Dependents' column (convert '3+' to 3)
# First, handle non-numeric entries like '3+'
train_data['Dependents'] = train_data['Dependents'].replace('3+', '3')
test_data['Dependents'] = test_data['Dependents'].replace('3+', '3')


# Now, convert the Dependents column to integers 
train_data['Dependents'] = pd.to_numeric(train_data['Dependents'].replace('3+', '3'), errors='coerce')
test_data['Dependents'] = pd.to_numeric(test_data['Dependents'].replace('3+', '3'), errors='coerce')

In [17]:
# Check the types again to ensure the conversion worked
print(train_data.dtypes)

Loan_ID               object
Gender                 int64
Married                int64
Dependents             int64
Education              int64
Self_Employed          int64
ApplicantIncome        int64
CoapplicantIncome      int64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area          int64
dtype: object


In [18]:
# Step 5: Create a target column 'Loan_Status' based on Credit_History for training and testing data
# Assuming Credit_History = 1 means loan approved and Credit_History = 0 means loan not approved
train_data['Loan_Status'] = train_data['Credit_History'].apply(lambda x: 1 if x == 1 else 0)
test_data['Loan_Status'] = test_data['Credit_History'].apply(lambda x: 1 if x == 1 else 0)


In [22]:
# If Loan_Status is categorical, encode it
y_train = y_train.map({'N': 0, 'Y': 1})
y_test = y_test.map({'N': 0, 'Y': 1})


In [23]:
# Step 7: Define feature variables (X) and target variable (y) for both training and testing sets
X_train = train_data.drop(['Loan_Status', 'Loan_ID'], axis=1)
y_train = train_data['Loan_Status']

X_test = test_data.drop(['Loan_Status', 'Loan_ID'], axis=1)
y_test = test_data['Loan_Status']

In [24]:
print(X_train.dtypes) 

Gender                 int64
Married                int64
Dependents             int64
Education              int64
Self_Employed          int64
ApplicantIncome        int64
CoapplicantIncome      int64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area          int64
dtype: object


In [25]:
# Step 8: Feature scaling (important for KNN)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) 
X_test_scaled = scaler.transform(X_test)       

In [26]:
#Checking the shape and previewing of the first 10 rows of the scaled data
print(X_train.shape)  
print(X_train[:10])    

(367, 11)
   Gender  Married  Dependents  Education  Self_Employed  ApplicantIncome  \
0       1        1           0          0              0             5720   
1       1        1           1          0              0             3076   
2       1        1           2          0              0             5000   
3       1        1           2          0              0             2340   
4       1        0           0          1              0             3276   
5       1        1           0          1              1             2165   
6       0        0           1          1              0             2226   
7       1        1           2          1              0             3881   
8       1        1           2          0              0            13633   
9       1        0           0          1              0             2400   

   CoapplicantIncome  LoanAmount  Loan_Amount_Term  Credit_History  \
0                  0       110.0             360.0             1.0   
1 

In [27]:
# Step 9: Cross-Validation (Logistic Regression and KNN)
# Logistic Regression Model
logreg = LogisticRegression(max_iter=500, solver='lbfgs')

In [28]:
# K-Fold Cross-Validation for Logistic Regression (k=5)
logreg_scores = cross_val_score(logreg, X_train_scaled, y_train, cv=5, scoring='accuracy')
print(f'Logistic Regression Cross-Validation Accuracy Scores: {logreg_scores}')
print(f'Logistic Regression Mean Accuracy: {logreg_scores.mean() * 100:.2f}%')


Logistic Regression Cross-Validation Accuracy Scores: [1. 1. 1. 1. 1.]
Logistic Regression Mean Accuracy: 100.00%


In [46]:
# KNN Model
knn = KNeighborsClassifier(n_neighbors=5)
# K-Fold Cross-Validation for KNN (k=5)
knn_scores = cross_val_score(knn, X_train_scaled, y_train, cv=5, scoring='accuracy')
print(f'KNN Cross-Validation Accuracy Scores: {knn_scores}')
print(f'KNN Mean Accuracy: {knn_scores.mean() * 100:.2f}%')

KNN Cross-Validation Accuracy Scores: [0.98648649 1.         1.         1.         1.        ]
KNN Mean Accuracy: 99.73%


In [47]:
logreg = LogisticRegression(max_iter=2000)  # Increase to 2000 iterations
logreg.fit(X_train_scaled, y_train)


LogisticRegression(max_iter=2000)

In [48]:
from sklearn.preprocessing import StandardScaler

# Scale your feature variables
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

logreg = LogisticRegression(max_iter=2000)
logreg.fit(X_train_scaled, y_train)

LogisticRegression(max_iter=2000)

In [49]:
#Penalising large coefficients with Regularisation
logreg = LogisticRegression(max_iter=2000, solver='liblinear', penalty='l2')
logreg.fit(X_train_scaled, y_train)

LogisticRegression(max_iter=2000, solver='liblinear')

In [50]:
# Convert back to DataFrame with original feature names
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

In [51]:
# Step 2: Train the Logistic Regression model
logreg = LogisticRegression(max_iter=2000, solver='liblinear')
logreg.fit(X_train_scaled_df, y_train)

LogisticRegression(max_iter=2000, solver='liblinear')

In [52]:
#Make predictions using the scaled DataFrame (with valid feature names)
y_pred = logreg.predict(X_test_scaled_df)

In [53]:
# K-Fold Cross-Validation for Logistic Regression (k=5)
logreg_scores = cross_val_score(logreg, X_train_scaled, y_train, cv=5, scoring='accuracy')
print(f'Logistic Regression Cross-Validation Accuracy Scores: {logreg_scores}')
print(f'Logistic Regression Mean Accuracy: {logreg_scores.mean() * 100:.2f}%')

Logistic Regression Cross-Validation Accuracy Scores: [1. 1. 1. 1. 1.]
Logistic Regression Mean Accuracy: 100.00%


In [54]:
# Tuning K for KNN (finding the optimal k value using cross-validation)
k_values = list(range(1, 21))  # Checking k values from 1 to 20
cv_scores = []

In [55]:
for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_train_scaled, y_train, cv=5, scoring='accuracy')
    cv_scores.append(scores.mean())

In [56]:
# Find the value of k with the highest accuracy
optimal_k = k_values[np.argmax(cv_scores)]
print(f'The optimal value of k is {optimal_k}')

The optimal value of k is 4


In [57]:
# Step 10: Train the KNN model on the training set
knn = KNeighborsClassifier(n_neighbors=4) 
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=4)

In [58]:
# Step 11: Test both models on the separate testing set and print their accuracy
y_pred_logreg = logreg.predict(X_test)
y_pred_knn = knn.predict(X_test)

logreg_accuracy = accuracy_score(y_test, y_pred_logreg)
knn_accuracy = accuracy_score(y_test, y_pred_knn)

In [59]:
# Step 11: Test both models on the test set and print their predictions
y_pred_logreg = logreg.predict(X_test)
y_pred_knn = knn.predict(X_test)

In [60]:
# Logistic Regression Evaluation
print("Logistic Regression Performance:")
print(f'Accuracy: {accuracy_score(y_test, y_pred_logreg) * 100:.2f}%')
print(f'Precision: {precision_score(y_test, y_pred_logreg) * 100:.2f}%')
print(f'Recall: {recall_score(y_test, y_pred_logreg) * 100:.2f}%')
print(f'F1 Score: {f1_score(y_test, y_pred_logreg) * 100:.2f}%')

Logistic Regression Performance:
Accuracy: 83.65%
Precision: 84.25%
Recall: 99.03%
F1 Score: 91.04%


In [61]:
# KNN Evaluation
print("\nKNN Performance:")
print(f'Accuracy: {accuracy_score(y_test, y_pred_knn) * 100:.2f}%')
print(f'Precision: {precision_score(y_test, y_pred_knn) * 100:.2f}%')
print(f'Recall: {recall_score(y_test, y_pred_knn) * 100:.2f}%')
print(f'F1 Score: {f1_score(y_test, y_pred_knn) * 100:.2f}%')


KNN Performance:
Accuracy: 86.38%
Precision: 91.08%
Recall: 92.86%
F1 Score: 91.96%


In [45]:
#to view confusion matrix as well
from sklearn.metrics import confusion_matrix
cm_logreg = confusion_matrix(y_test, y_pred_logreg)
cm_knn = confusion_matrix(y_test, y_pred_knn)
print("Confusion Matrix for Logistic Regression:\n", cm_logreg)
print("Confusion Matrix for KNN:\n", cm_knn)


Confusion Matrix for Logistic Regression:
 [[  2  57]
 [  3 305]]
Confusion Matrix for KNN:
 [[ 31  28]
 [ 22 286]]
